# Background

Sebagai data scientist, tugas utama kita akan selalu bergelut dengan data. Namun, selama ini, apakah kita sadar darimana data kita berasal? bagaimana data kita dikumpulkan? dsb dsb? Untuk menjadi seorang data scientist yang baik, saya rasa pengetahuan dan pengalmaan dalam mengumpulkan data cukup penting. Apakah anda pernah kesulitan dalam mencari data? apakah anda pernah kesulitan dalam melakukan pelabelan data? apakah kalian percaya bahwa data seharusnya gratis dan mudah untuk diakses. It's Data for Democracy!

Untuk membuat data dapat diakses oleh semua orang, pihak penyedia jasa juga harus memberikan batasan guna mengurangi penggunaan berlebihan dan penyalahgunaan data oleh pihak-pihak yang tidak bertanggungjawab. Untuk menanggulangi permasalahn ini, kita bisa memanfaatkan teknologi API (Application Program Interface). Dengan API, kita dapat mengatur bagaimana data kita dapat diakses secara umum.

Dalam Capstone kali ini, kita akan mencoba membuat API menggunakan python + flask agar data kita dapat diakses secara umum. Secara konsep, kita akan membangun aplikasi python menggunakan flask yang dapat mengatur, membaa, dan mengirimkan response terhadap request user.


**Data yang digunakan:** 
- books_c.csv

**Environtments:**
- python 
- pandas
- flask 
- gunicorn


**Goals**
1. Berhasil membuat Flask APP yang berfungsi sebagai API yang memberikan data dalam format JSON
2. Berhasil membuat minimal 2 endpoint statis (atau lebih) dan 1 endpoint dinamis(atau lebih) menggunakan routing
3. Berhasil melakukan deployment Flask APP ke Heroku

*Notes: menggunakan endpoints yang sudah dicontohkan tidak akan dihitung sebagai endpoint hasil kerja capstone*

# Membangun API Python dalam 6 menit 
*Disclaimer: Course ini adalah course singkat untuk membantu kalian dalam membuat API. Akan ada sangat banyak kekurangan dari konsep API yang ada saat ini. Namun untuk kemudahan, kita hanya akan membahas konsep dasar dari API dan membuat Flask App sederhana yang akan bertugas sebagai API

Kita akan mencoba membangun Flask APP sebagai API, oleh karena itu jika kalian belum memiliki libaray Flask, silakan install menggunakan `pip install flask`. Berikut adalah beberapa library yang akan kita butuhkan. Cobalah import library tersebut sebelum menginstallnya. 

In [7]:
# !pip install flask
# !pip install pandas
# !pip install requests

In [8]:
import pandas as pd

In [9]:
import flask

In [10]:
import requests

In [11]:
import gunicorn

Membuat aplikasi Flask sesederhana menulis code dalam 7 baris !
cobalah buat file `app.py` yang berisi script berikut :

In [12]:
from flask import Flask, request 
app = Flask(__name__) 

@app.route('/home')
def home():
    return 'Hello World'

if __name__ == '__main__':
    app.run(debug=True, port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

Setelah membuat file `app.py` berisikan script diatas, cobalah jalankan file tersebut melalui terminal dengan masuk ke direktori yang sama dengan file tersebut, dan jalankan sintaks `python app.py`. Setelah menjalankan sintaks tersebut, maka aplikasi kita akan berjalan dengan alamat `localhost` dengan port `5000`, atau dapat kalian akses melalui browser dengan alamat `localhost:5000`. 

pada baris ke-4, terdapat sintaks `@app.route('/home')`, yang berarti bahwa fungsi atau method dibawahnya (fungsi `home`) akan dijalankan ketika user mengakses rute atau alaman `/home`. Sekarang, cobalah akses `localhost:5000/home`, maka kalian akan mendapatkan tulisan 'Hello World'

Pada baris terakhir, `debug=True` akan menampilkan semua log aktivitas dari app kita dalam terminal/comman prompt/shell dimana app kita dijalankan. 

Dari contoh script diatas, kita dapat katakan bahwa rute atau `route` adalah endpoints, atau tempat dimana sistem dan user saling bersentuhan. Dalam kasus diatas, untuk mendapatkan 'Hello World', maka kita harus mengakses endpoint `/home`. Sekali lagi, **representasi endpoint adalah route**.

## Query Arguments

Setelah mengetahui kerangka utama dalam membuat aplikasi Flask, cukup penting untuk mengetahui Query Arguments. Ketika kita melakukan pencarian pada mesin pencari seperti google.com, jika kita perhatikan pada alamat (url) yang diberikan, terdapat query arguments yang diberikan setelah `google.com/` yang diawali dengan `?`. 

Format query pada url pada ummnya adalah `example.com?arg1=value1&arg2=value2`, dimana query akan diawali dengan `?`, dihubungkan dengan `&`, dan setiap query adalah pasangan `key` dan `value`

In [ ]:
from flask import Flask, request 
app = Flask(__name__) 

@app.route('/query')
def query_example():
    key1 = 'name'
    key2 = 'age'
    name = request.args[key1] # Jika key tidak disertakan dalam URL, maka akan terjadi server error
    age = request.args.get(key2) # Jika key tidak disertakan dalam URL, maka age akan bernilai None
    return(f"Hello, {name}, yoau are {age} years old")

if __name__ == '__main__':
    app.run(debug=True, port=5000) 

Silakan update file `app.py` dengan script diatas, lalu akses `localhost:5000/query?name=Budi Setiawan`. Perhatikan error yang ditampilkan. Sekarang, cobalah akses  `localhost:5000/query?name=Budi Setiawan&age=32`. Lalu bandingkan ketika kita mengakses  `localhost:5000/query?age=32`. Dari contoh diatas, terdapat dua cara untuk mendaptkan arguments dari url yang dikirim oleh user. Pertama adalah dengan menggunakan `request.args[key]`. Namun cara ini cukup beresiko mengingat jika key tidak tersedia dalam url, maka akan terjadi server error. Cara pertama ini jarang digunakna kecuali jika kita memang mewajibkan adanya argumen tertentu. Cara yang kedua adalah `request.args.get(key)`. Cara kedua ini lebih sering digunakan karena secara otomatis akan memberikan nilai None jika key tidak terdapat dalam url.

Dalam API, penggunaan query arguments dapat sangat membantu terutama dalam endpoint dinamis. Kita akan pelajari bagaimana query arguments akan sangat membantu ktia dalam menyederhanakan pembuatan API server kita. 

## API Methods

Apakah kalian familiar dengan **CRUD** (**C**reate, **R**read, **U**pdate, **D**elete) dalam database? Dalam API, konsep tersebut diadopsim menjadi 
- POST(CREATE), 
- GET(READ), 
- PUT(UPDATE), 
- DELETE

Secara konsep, method POST, GET, PUT, dan DELETE dibuat untuk mengatur API agar dapat lebih terstruktur. Meskipun pada prakterknya, endpoint yang kita definisikan menggunakan method "POST" tidaklah harus untuk membuat data. Artinya, tidak ada aturan yang mengharuskan kita untuk menuliskan jenis method dalam setiap endpoints kita (default endpoints menggunakan method GET). Namun, untuk tujuan kerapihan dan mengikuti kaidah yang benar, kita akan coba mendefinisikan endpoints kita dengan method tertentu, dan melihat manfaat dari penggunaan jenis method dalam setiap endpoint. 



In [ ]:
from flask import Flask, request 
app = Flask(__name__)

@app.route('/form', methods=['GET', 'POST']) #allow both GET and POST requests
def form():
    if request.method == 'POST':  # Hanya akan tampil setelah melakukan POST (submit) form
        key1 = 'name'
        key2 = 'age'
        name = request.form.get(key1)
        age = request.form[key2]
        
        return (f'''<h1>Your Name  is: {name}</h1>
                   <h1>Your Age is: {age}</h1>
                ''')

    return '''<form method="POST">
                  Name: <input type="text" name="name"><br>
                  Age: <input type="text" name="age"><br>
                  <input type="submit" value="Submit"><br>
              </form>'''

if __name__ == '__main__':
    app.run(debug=True, port=5000) 

Silakan update file `app.py` kalian dengan sintaks diatas, lalu akses `localhost:5000/form`, aka akan muncul form sederhana dengan tombol "submit". Mengapa form tampil terlebih dahulu? karena dalam url `localhost:5000/form`, kita tidak mendefinisikan method kita sebagai POST atau GET. Maka, secara default request kita akan dianggap sebagai GET. Apakah kita bisa menentukan method kita dalam url? Kami sarankan untuk tidak mencobanya. 

Dalam sintaks diatas, kita mendefinisikan rute atau endpoint `/form` sebagai GET atau POST. Hal ini berarti endpoint tersebut dapat menerima request GET atau POST. 

Setelah kita mengisi form dan menekan tombol "submit", maka kita akan melakukan request POST dengan mengirimkan data-data berupa nama dan umur kepada server kita. Oleh sebab itu, server kita akan menjalankan perintah yang berada dalam kondisi `if request.method == 'POST':`

Cobalah untuk mengganti `if request.method == 'POST':` menjadi `if request.method == 'GET':`, maka kalian akan melihat perbedannya. 

Jika kalian penasaran, cobalah ubah `@app.route('/form', methods=['GET', 'POST'])` menjadi `@app.route('/form', methods=['POST'])`


## JSON Data Type

JSON adalah standar tipe data dalam pengiriman data dengan protokol HTTP. Pada dasarnya, JSON (JavaScript Object Name) sangat mirip, atau bisa dibilang identik dengan tipe data `dictionary` dalam python. Dictionary terdiri dari pasangan `key`:`value`. Berikut adalah contoh bentuk dan akses data dictionary. 

In [ ]:
books = {'author': 'Anthony',
         'num_of_page': 123,
         'year' : 2019
        }
print(f"Pengaran buku: {books['author']}, dengan halaman {books['num_of_page']}, diterbitkan pada {books['year']}")

Kita akan mencoba mengirimkan data JSON menggunakan method POST ke endpoint kita. Dikarenakan browser biasa tidak bisa menangani hal tersebut, kita bisa gunakan [postman](https://www.getpostman.com/) atau jupyter notebook untuk membantu.
Silakan update file `app.py` dengan script berikut: 

In [ ]:
from flask import Flask, request 
app = Flask(__name__) 

@app.route('/json', methods=['POST']) 
def json_exmp():
    
    req = request.get_json(force=True) # melakukan parsing data json, menyimpannya sebagai dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')

if __name__ == '__main__':
    app.run(debug=True, port=5000) #run app in debug mode on port 5000

Setelah server tersebut diupdate, kita akan coba melakukan request menggunakan library requests pada python

In [ ]:
import requests

url = 'http://localhost:5000/json'
data = {"name" : "Andi", 
        "age" : 10,
        "address" : "Jakarta"
       }
r = requests.post(url,json=data) # pastikan gunakan requests.pos, bukan requests.get
print(r)

Response 200 berarti API kita berhasil mengirimkan atau melakukan request yang kita kirim. Untuk mempelajari jenis-jenis repsonse yang dikirimkan melalui HTTP, silakan merujuk ke [sumber ini](https://restfulapi.net/http-status-codes/). Objek yang kita kirimkan berupa `requests.models.Response`. Untuk mendapatkan isi dari respon tersebut, kita dapat gunakan `.text` atau `.content`

In [ ]:
print(type(r))

In [ ]:
print(r.content)
print(r.text)

## Data Fetching

Kembali ke tujuan utama, kita akan mencoba membuat API yang mampu mengirimkan data dalam bentuk DataFrame (dalam format JSON). Dalam hal ini, kita akan coba menggunakan data `books_c.csv` yang terdapat dalam folder `data_input`. Dalam contoh kali ini, kita akan mengaplikasikan dynamic routing. Konsepnya adalah kita tidak secara eksplisit membuat route atau endpoint, melainkan menyimpannya dalam variabel. Hal ini akan sangat berguna jika data yang ingin kita berikan sangat beragam dan bergantung pada keinginan user.

Untuk melakukan dynamic routing, cukup simpan variabel kedalam tanda `< >`, lalu masukkan variabel tersebut sebagai parameter fungsi di endpoint yang bersangkutan. 

In [44]:
from flask import Flask, request 
import pandas as pd 
import sqlite3
app = Flask(__name__) 

# mendapatkan keseluruhan data dari <data_name>
@app.route('/data/get/<data_name>', methods=['GET']) 
def get_data(data_name): 
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("SELECT firstname, lastname, email, company, \
                                 invoiceid, invoicedate, billingcountry, total \
                                 FROM invoices \
                                 left join customers \
                                 on invoices.customerId = customers.customerId\
                                 ", conn)
    return (data.to_json())

@app.route('/data/get/artist/<data_name>/<name>', methods=['GET']) 
def get_artistName(data_name, name): 
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("SELECT tracks.*, albums.Title, genres.Name AS GenreName,\
                            artists.Name AS ArtistName \
                            FROM tracks \
                            LEFT JOIN albums \
                            ON albums.AlbumId=tracks.AlbumId \
                            LEFT JOIN genres \
                            ON genres.GenreId=tracks.GenreId \
                            LEFT JOIN artists \
                            ON artists.ArtistId=albums.ArtistId ", conn, index_col='TrackId')
    
    mask = data['ArtistName'] == name
    data = data[mask]
    return (data.to_json())

@app.route('/data/get/genre/<data_name>/<value>', methods=['GET']) 
def get_genre(data_name, value): 
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("SELECT tracks.*, albums.Title, genres.Name AS GenreName,\
                            artists.Name AS ArtistName \
                            FROM tracks \
                            LEFT JOIN albums \
                            ON albums.AlbumId=tracks.AlbumId \
                            LEFT JOIN genres \
                            ON genres.GenreId=tracks.GenreId \
                            LEFT JOIN artists \
                            ON artists.ArtistId=albums.ArtistId ", conn, index_col='TrackId')

    mask = data['GenreName'] == value
    data = data[mask]
    return (data.to_json())

# mendapatkan data dengan filter nilai <value> pada kolom <column>
@app.route('/data/get/email/<data_name>/<email>', methods=['GET']) 
def get_email(data_name, email): 
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("SELECT firstname, lastname, email, company \
                                 invoiceid, invoiceDate , billingcountry, total \
                                 FROM invoices \
                                 left join customers \
                                 on invoices.customerId = customers.customerId \
                                 ", conn)
    mask = data['Email'] == email
    data = data[mask]
    return (data.to_json())

@app.route('/data/get/track/<data_name>', methods=['GET']) 
def get_track(data_name): 
    conn = sqlite3.connect("data/chinook.db")
    data = pd.read_sql_query("SELECT c.Country, c.FirstName, c.LastName, invoices.Total, \
                           invoices.InvoiceDate \
                           FROM customers as c \
                           LEFT JOIN invoices \
                           ON c.CustomerId = invoices.CustomerId" ,conn)
    return (data.to_json())

if __name__ == '__main__':
    app.run(debug=True, port=5000) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\leona\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Untuk meminta data menggunakan API yang telah ada, silakan jalankan cells berikut

In [18]:
import requests
import pandas as pd

In [57]:
url1 = 'http://localhost:5000/data/get/chinook.db'
r = requests.get(url1)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Leonie,Köhler,leonekohler@surfeu.de,None,1,2009-01-01 00:00:00,Germany,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2,2009-01-02 00:00:00,Norway,3.96
2,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
3,Mark,Philips,mphilips12@shaw.ca,Telus,4,2009-01-06 00:00:00,Canada,8.91
4,John,Gordon,johngordon22@yahoo.com,None,5,2009-01-11 00:00:00,USA,13.86


In [58]:
url2 = 'http://localhost:5000/data/get/artist/chinook.db/Accept'
r = requests.get(url2)
r_pd = pd.DataFrame(r.json())
r_pd

,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,GenreName,ArtistName
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,Rock,Accept
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Restless and Wild,Rock,Accept
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Restless and Wild,Rock,Accept
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Restless and Wild,Rock,Accept


In [59]:
url3 = 'http://localhost:5000/data/get/genre/chinook.db/Jazz'
r = requests.get(url3)
r_pd = pd.DataFrame(r.json())
r_pd.tail()

,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,GenreName,ArtistName
2530,Little Linda,204,1,2,Jeremy Wall,264019,8958743,0.99,Morning Dance,Jazz,Spyro Gyra
2531,End Of Romanticism,204,1,2,Rick Strauss,320078,10553155,0.99,Morning Dance,Jazz,Spyro Gyra
3349,Amanda,262,5,2,Luca Gusella,246503,4011615,0.99,Quiet Songs,Jazz,Aisha Duo
3350,Despertar,262,5,2,Andrea Dulbecco,307385,4821485,0.99,Quiet Songs,Jazz,Aisha Duo
3357,OAM's Blues,267,5,2,Aaron Goldberg,266936,4292028,0.99,Worlds,Jazz,Aaron Goldberg


In [60]:
url4 = 'http://localhost:5000/data/get/email/chinook.db/bjorn.hansen@yahoo.no'
r = requests.get(url4)
r_pd = pd.DataFrame(r.json())
r_pd.tail()

,FirstName,LastName,Email,invoiceid,InvoiceDate,BillingCountry,Total
75,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2009-11-25 00:00:00,Norway,0.99
196,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2011-05-19 00:00:00,Norway,1.98
207,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2011-06-29 00:00:00,Norway,15.86
262,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2012-02-27 00:00:00,Norway,8.91
391,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2013-10-03 00:00:00,Norway,1.98


In [61]:
url5 = 'http://localhost:5000/data/get/track/chinook.db'
r = requests.get(url5)
r_pd = pd.DataFrame(r.json())
r_pd.tail()

,Country,FirstName,LastName,Total,InvoiceDate
407,India,Puja,Srivastava,5.94,2009-07-08 00:00:00
408,India,Puja,Srivastava,1.99,2010-02-26 00:00:00
409,India,Puja,Srivastava,1.98,2011-08-20 00:00:00
410,India,Puja,Srivastava,13.86,2011-09-30 00:00:00
411,India,Puja,Srivastava,8.91,2012-05-30 00:00:00


Dari contoh diatas, kita sudah dapat melakukan fetching data menggunakan API dan seberapa bermanfaatnya melakukan dynamic routing. Kedepannya, diharapkan pengetahuan ini dapat dikembangkan dengan melakukan fetching data dari database. 

## Deploying Flask APP 

Sekarang, setelah Flask App kita sudah berhasil dijalankan, kita akan coba deploy app tersebut **(beserta environmentnya)** ke app hosting/web hosting/cloud. Untuk saat ini, kita akan menggunakan [heroku](https://www.heroku.com/) karena **gratis**. Untuk melakukan deployment, silakan registrasi akun baru. 

Langkah ini dapat kalian ikuti pada [sumber ini](https://stackabuse.com/deploying-a-flask-application-to-heroku/)

### Get the requirements

Silakan buat environment baru, lalu hanya install `pandas`, `flask`, dan `gunicorn`. Setelah libary tersebut berhasil diinstal, jalankan `pip freeze > requirements.txt` untuk mengeksport semua library kedalam file `requirements.txt`. Perlu diingat bahwa Heroku hanya bisa mengenali libary yang kita pakai jika kita menuliskannya kedalam requirements.txt

### Create Heroku App
Untuk membuat heroku app, cukup klik new->app pada dashboard heroku. Perlu diingat bahwa nama app di heroku haruslah unik. 

### Deployment

Setelah kita membuat dan menamai app kita di heroku, kita harus mengupload file app kita dari lokal. Untuk melakukan hal ini, kita akan menggunakan git. Cara termudah adalah dengan membuat repository baru di github, lalu sambungkan aplikasi heroku degan repository github tersebut. 
![](res/deploy-1.PNG)

Nyalakan `Automatic Deploy` jika ingin melakukan deployment ulang setiap terjadi perubahan pada repositoy secara otomatis (kurang disarankan). Setelah repository terhubung, klik tombol deploy branch yang berada di bagian bawah app heroku (jika tidak ada branch pada repository github, secara default branch master yang akan dipilih). 

![](res/deploy-2.PNG)

# Test Your API Endpoints

Setelah deployment app kita behasil, kita bisa mencoba mengaksesnya melalui browser, atau jupyter notebook(lebih disarankan). Mari coba beberapa endpoints yang telah kita coba lakukan di lokal. 

Untuk catatan, tidak perlu menuliskan port pada url heroku, karena kita sudah mengaturnya pada file `Procfile`

In [4]:
import requests
import pandas as pd 
heroku_url = 'https://algo-capstone.herokuapp.com/data/get/books_c.csv'
r = requests.get(heroku_url)
r_pd = pd.DataFrame(r.json())
r_pd

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.55,043965548X,9780439655484,eng,435,2149872,33964
...,...,...,...,...,...,...,...,...,...,...
13709,47699,M Is for Magic,Neil Gaiman-Teddy Kristiansen,3.82,0061186422,9780061186424,eng,260,11317,1060
13710,47700,Black Orchid,Neil Gaiman-Dave McKean,3.72,0930289552,9780930289553,eng,160,8710,361
13711,47701,InterWorld (InterWorld #1),Neil Gaiman-Michael Reaves,3.53,0061238961,9780061238963,en-US,239,14334,1485
13712,47708,The Faeries' Oracle,Brian Froud-Jessica Macbeth,4.43,0743201116,9780743201117,eng,224,1550,38


# Document your API

Hal terpenting dalam pembuatan API adalah dokumentasi. Menjadi pembangun aplikasi yang handal akan percuma jika user tidak tahu cara menggunakannya. Hal ini juga memudahkan kami untuk menginspeksi dan menilai captsone kalian. Oleh karena itu, kita akan mencoba membuat dokumentasi API sederhana dengan format : 
- Deskripsi Utama 
- Endpoints
___
```
Deskripsi: 
API ini dinominasikan sebagai capstone project yang berguna untuk mengirimkan data kepada user. Proses wrangling dilakukan sesuai endpoint-endpoint yang dimaksud. Base url dari aplikasi ini adalah https://algo-capstone.herokuapp.com

Endpoints: 
    1. / , method = GET
    Merupakan endpoint home, dan akan mengembalikan nilai berupa string selamat datang
    
    2. /data/get/<data_name> , method = GET
    Mengembalikan data <data_name> dalam bentuk JSON. Beberapa data yang tersedia adalah : 
        - books_c.csv
        - pulsar_stars.csv (tidak digunakan, hanya contoh saja)
        
    3. /data/get/equal/<data_name>/<column>/<value>
    Mengembalikan <data_name> yang telah di filter dimana nilai pada <column> = <value>

```
___

# Tips 

Jika kalian merasa sedikit kesulitan untuk menentukan apa informasi yang akan dikirimkan dalam sebuah endpoint (membuat endpoint), cobalah beralih ke sudut pandang user. Sebagai user, informasi apa yang ingin kita dapatkan dari data tersebut? Akan ada sangat banyak kemungkinan yang bisa diambil. Kalian bisa melakukannya! 

Happy Coding !
~ Team Algoritma